<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 521 Files and Databases - Fall 2016</h1>
<h1 style="font-size: 250%;">Problem Set #4</h1>
<h1 style="font-size: 200%;">Version 1.0</h1>
<h1>Issued Wednesday 11/02/2016; Due Monday 11/14/2016</h1>
</div>

**Homework Information:** The answer cells in this notebook contain metadata that enable it to be automatically parsed. Thus, you must enter your answers *only* into the cells provided. Do not delete any answer cell and/or add a new one to replace it. If you do so by accident, download a new copy of the problem set's notebook and transfer your answers into it. You can introduce new 'coding' cells in an effort to test your approaches to problems, however, these introduced cells will not be considered when grading.

Finally, some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will be penalized by a factor of 70.71% for each class meeting after their due date. Feel free to get help from others, but the work you submit in should be your own.

In [1]:
# Fill in the fields as described
class Student:
    first = "James"
    last = "Barbour"
    onyen = "jebarbou"
    pid = "720076615"

---
**Problem 1:** Using the <a href="http://csbio.unc.edu/mcmillan/Media/movies.db" download>movies</a> database from problem set #2, write a Python function to print filmographies of all of the actors from a given **movie title**. The format of your output should be:

    Actor1.name,
       Movie1.title, Movie1.release
       Movie2.title, Movie2.release
       Movie3.title, Movie3.release
       Movie4.title, Movie4.release
           .
           .
           .
    Actor2.name,
       Movie1.title, Movie1.release
       Movie2.title, Movie2.release
       Movie3.title, Movie3.release
       Movie4.title, Movie4.release
           .
           .
           .

Where Actor1, Actor2, are all members of the cast in the given **movie title**, which should appear in all of their filmographies.

In [3]:
import sqlite3, pandas


def printfilmography(movie):
    conn = sqlite3.connect('movies.db')
    actors = pandas.read_sql_query(
            'SELECT DISTINCT A.aid, A.name \
            FROM Actors A\
            WHERE A.aid IN ( \
                SELECT C.aid \
                FROM Cast C \
                WHERE C.mid IN ( \
                    SELECT M.mid \
                    FROM Movies M \
                    WHERE M.title = "' + movie + '"))', conn)
    for aid, aname in actors.values:
        print(aname + ',')
        for title, release in pandas.read_sql_query(
                'SELECT DISTINCT title, release \
                FROM Movies JOIN \
                    (SELECT C.aid, C.mid \
                    FROM Cast C \
                    WHERE C.aid = "' + str(aid) + '") \
                    USING(mid) \
                WHERE aid = "' + str(aid) + '"', conn).values:
            print('   ' + title + ', ' + release)

---
** Problem 2:** Use Jupyter's [*%timeit*](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) function to time the function that you wrote for the movie "Blade Runner".

In [5]:
%timeit[printfilmography('Blade Runner')]

Harrison Ford,
   Star Wars, 1977-03-20
   Apocalypse Now, 1979-08-15
   Blade Runner, 1982-06-25
   Raiders of the Lost Ark, 1981-06-12
   Indiana Jones and the Temple of Doom, 1984-05-23
   Indiana Jones and the Last Crusade, 1989-05-24
   Indiana Jones and the Kingdom of the Crystal Skull, 2008-05-21
   The Conversation, 1974-04-07
   American Graffiti, 1973-08-01
   The Empire Strikes Back, 1980-01-01
   Return of the Jedi, 1983-05-23
   What Lies Beneath, 2000-07-21
   Zabriskie Point, 1970-02-09
   Working Girl, 1988-12-20
   The Devil's Own, 1997-03-12
   A Time for Killing, 1967-08-01
   The Fugitive, 1993-08-06
   Six Days Seven Nights, 1998-06-12
   K-19: The Widowmaker, 2002-07-19
   Witness, 1985-02-08
   Clear and Present Danger, 1994-08-03
   Firewall, 2006-02-10
   Air Force One, 1997-07-23
   Patriot Games, 1992-06-04
   Frantic, 1988-02-16
   Presumed Innocent, 1990-07-26
   The Mosquito Coast, 1986-11-26
   Hollywood Homicide, 2003-06-09
   Sabrina, 1995-12-15
   Cros

---
**Problem 3:** Write Python code to build an index on the Cast relation of movies.db with the search key (aid, mid). Then rerun your function from Problem #2 for "Blade Runner" using *%timeit*.

In [6]:
conn = sqlite3.connect('movies.db')
curs = conn.cursor()
curs.execute('CREATE INDEX castidx ON Cast (aid, mid)')
conn.commit()
conn.close()
%timeit[printfilmography("Blade Runner")]

Harrison Ford,
   Star Wars, 1977-03-20
   Apocalypse Now, 1979-08-15
   Blade Runner, 1982-06-25
   Raiders of the Lost Ark, 1981-06-12
   Indiana Jones and the Temple of Doom, 1984-05-23
   Indiana Jones and the Last Crusade, 1989-05-24
   Indiana Jones and the Kingdom of the Crystal Skull, 2008-05-21
   The Conversation, 1974-04-07
   American Graffiti, 1973-08-01
   The Empire Strikes Back, 1980-01-01
   Return of the Jedi, 1983-05-23
   What Lies Beneath, 2000-07-21
   Zabriskie Point, 1970-02-09
   Working Girl, 1988-12-20
   The Devil's Own, 1997-03-12
   A Time for Killing, 1967-08-01
   The Fugitive, 1993-08-06
   Six Days Seven Nights, 1998-06-12
   K-19: The Widowmaker, 2002-07-19
   Witness, 1985-02-08
   Clear and Present Danger, 1994-08-03
   Firewall, 2006-02-10
   Air Force One, 1997-07-23
   Patriot Games, 1992-06-04
   Frantic, 1988-02-16
   Presumed Innocent, 1990-07-26
   The Mosquito Coast, 1986-11-26
   Hollywood Homicide, 2003-06-09
   Sabrina, 1995-12-15
   Cros

---
Suppose that a certain relation has 250,000 records and a B<sup>+</sup>-tree index is built on it with 24 keys per node, both leaf and interior.

**Problem 4:** Assume that the records are loaded using a bulk-loader that fills each node, with the possible exception of the root, to at least 2/3 of its capacity. How many levels will be in the final tree? How many interior *steering* nodes will the tree have?

**Problem 5:** Assume that the records were loaded via a series of insertions and deletions that resulted in a B<sup>+</sup>-tree where as many nodes as possible arefilled to their maximum capacity. How many levels will be in this tree? How many interior *steering* nodes will this tree have?

**Problem 6:** Assume that the records were loaded via a series of insertions 
and deletions that resulted in a B<sup>+</sup>-tree where <i>every</i> node is filled to its minimum capacity. How many levels will be in this tree? How many interior *steering* nodes will this tree have?</p>


Consider the tree shown below:

<div align="center"><img title="comp521f10pset4p1tree.png" src="http://csbio.unc.edu/mcmillan/Media/comp521PS4Tree.png" width="509" height="109" />
</div>

**Problem 7:** Suppose this is an ISAM tree.
How many leaf nodes will have overflow pages after inserting the
following elements: (16, 17, 18) and then deleting the elements: (11, 12, 16)?</p>


**Problem 8:** Suppose this is a B+ tree.
Consider the resulting tree after inserting the following elements (16, 17, 18).
What will be the contents of the right-most non-leaf (interior) node?

---
Use the following simple hashing function, key = aid % 1024, to build indexes
for the *Actors* and *Cast* tables of the movies.db database using Python.
Use your indexes to answer the following questions.

**Problem 9:** What is the occupancy of (number of records in) the largest, smallest, and average bucket of the *Actors* hash index *aid*?</p>

**Problem 10:** What is the occupancy of (number of records in) the largest, smallest, and average bucket of the *Cast* hash index on *aid*?</p>

**Problem 11:** Using your indices estimate the number of tests used when joining Actors and Roles in a query such as the following:</p>
<pre>
    SELECT A.name, C.role
    FROM Actors A, Cast C
    WHERE A.aid=R.aid
</pre>

**Problem 12:** Of the two relations, *Actors* and *Cast* which would benefit more by using a dynamic hashing scheme? Explain why.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set. **Make sure that you upload it wiht the correct problem set number!**

You can submit it as many times as you would like, but only the last submitted version will be graded. If the last submission was after the due date the final score will be reduced accordingly. We do not need your database, since your Notebook should produce it.